In [ ]:
import json
import time
from datetime import date
import pprint
from os import listdir
from feature_extract import feature_extract

HOUR_INDEX = 3
MINUTE_INDEX = 4
SECOND_INDEX = 5
    
files = listdir('../tweet_data')
print(files)

from os.path import splitext

import re

for fname in files:
    m = re.search('#.+', splitext(files[0])[0])
    if m:
        hashtag = m.group()
        print("Processing '%s'" % hashtag)
    outfile = splitext(fname)[0] + '.csv'
    print("Output file name '%s'" % outfile)
    
    f_handle = open('../tweet_data/'+fname, encoding="utf8")
    feature_extract(f_handle, outfile)
    f_handle.close()
    break;

In [ ]:
import json
import time
from datetime import date

HOUR_INDEX = 3
MINUTE_INDEX = 4
SECOND_INDEX = 5

def feature_extract(f, o_file):
    import json
    import time
    from datetime import date

    line = f.readline()
    tweet = json.loads(line)

    n_tweets = 0 # number of tweets per hour
    n_followers = {} # uid -> (nfollower, ntweets) per hour
    n_retweets = 0 # number of retweets per hour
    num_window = 0 # total number of hours

    user_follower = {}

    # self defined features (for part 2)
    # n_users:-  number of users tweeted per hour
    # n_users_3:- number of users who tweeted 3 times or more per hour
    n_len_100 = 0 # number of tweets with more than 100 characters per hour

    # outputs
    import numpy as np

    """
    output: n_tweets  n_retweets  sum_followers  max_followers  hour \
            avg_tweet_per_user  n_users  n_users_3  n_len_100
    """
    output = np.empty([0, 9])

    # get start hour from first tweet
    start_time = list(time.localtime(tweet['firstpost_date']))
    start_time[MINUTE_INDEX] = 0; start_time[SECOND_INDEX] = 0
    nth_hour = start_time[HOUR_INDEX]
    start_time = time.mktime(start_time)
    start_date = date.fromtimestamp(start_time)
    print "Starting from time %s" % time.strftime("%Y-%m-%d %H:%M:%S", time.localtime(start_time))

    time_format = "%Y-%m-%d %H:%M:%S"

    while len(line) != 0:
        tweet = json.loads(line)
        line = f.readline()
        tweet_time = tweet['firstpost_date']
        tweet_hour = time.localtime(tweet_time).tm_hour
        if tweet_hour != nth_hour or start_date != date.fromtimestamp(float(tweet_time)):
            print "Finished processing %d tweets from hour %d, next date time %s" %\
                  (n_tweets, nth_hour, time.strftime(time_format, time.localtime(tweet_time)))
            # update output
            tweets_per_user = np.array(n_followers.values())[:, 1]
            new_hour = np.array([[n_tweets, n_retweets, sum(n_followers), max(n_followers), nth_hour, \
                                  np.mean(tweets_per_user), len(n_followers), sum(tweets_per_user >= 3), \
                                  n_len_100]])
            output = np.append(output, new_hour, axis=0)
            # clear & update hourly variables
            num_window += 1
            n_tweets = 0 # number of tweets per hour
            n_followers = {} # number of followers of users posting the tweets per hour
            n_retweets = 0 # number of retweets per hour
            nth_hour = tweet_hour  # hour of the day
            n_len_100 = 0

        # extract features
        n_tweets += 1
        n_retweets += tweet['tweet']['retweet_count']
        if tweet['tweet']['user']['id'] in n_followers:
            # increment user tweet count
            tweet_count = n_followers[tweet['tweet']['user']['id']][1] + 1
            n_followers[tweet['tweet']['user']['id']] = (tweet['tweet']['user']['followers_count'], tweet_count)
        else:
            n_followers[tweet['tweet']['user']['id']] = (tweet['tweet']['user']['followers_count'], 1)
        if (len(tweet['tweet']['text']) >= 100): n_len_100 += 1
        user_follower[tweet['tweet']['user']['id']] = tweet['tweet']['user']['followers_count']
        start_date = date.fromtimestamp(float(tweet_time))

    print "Outputting to '%s' ..." % o_file
    np.savetxt('data/' + o_file, output, delimiter=',')

    print "--------------------"
    print "Total number of tweets %d" % int(np.sum(output[:, 0]))
    print "Average number of tweets per hour %f" % np.mean(output[:, 0])
    print "Average number of retweets %f" % float(np.sum(output[:, 1]) / np.sum(output[:, 0]))
    print "Average number of followers of (%d) users %f" %\
          (len(user_follower), float(sum(user_follower.values()) / float(len(user_follower))))

In [8]:
import matplotlib.pyplot as plt
import numpy as np

history = np.loadtxt('data/#SuperBowlhistogram')
hours = np.loadtxt('data/#SuperBowlhistogramlabel')

plt.cla()
#plt.yscale('log')
plt.plot(hours, history)
plt.title('#SuperBowl Hourly Tweet Count')
plt.xlabel('Hour')
plt.ylabel('Number of Tweets')

plt.xticks(range(len(hours))[::100], hours[::100])
plt.savefig('graphs/#SuperBowl_tweet_count_histogram')
plt.show()

In [7]:
import matplotlib.pyplot as plt
import numpy as np

history = np.loadtxt('data/#NFLhistogram')
hours = np.loadtxt('data/#NFLhistogramlabel')

plt.cla()
#plt.yscale('log')
plt.plot(hours, history)
plt.title('#NFL Hourly Tweet Count')
plt.xlabel('Hour')
plt.ylabel('Number of Tweets')

plt.xticks(range(len(hours))[::100], hours[::100])
plt.savefig('graphs/#NFL_tweet_count_histogram')
plt.show()

In [ ]:
### Problem 2 and 3
from os import listdir
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression as LR
from sklearn.linear_model import LassoLars as lasso
from sklearn.linear_model import LogisticRegression as LogR
from sklearn.feature_selection import f_regression as freg
import numpy as np
import pandas as pd

# directory name
path = 'data/'

# hashtag name
hashtag = []

# csv file name
csv = []

files = listdir(path)

# append csv names into list
for f in files:
    if f.endswith(".csv"):
        hashtag.append(f[7:-4])
        csv.append(path + f)

# start the training per hashtag file

col_names = ["n_tweets", "n_retweets", "sum_followers", "max_followers", "hour", "avg_tweet_per_user", "n_users", "n_users_3", "n_len_100"]
for c in csv:
    print(c + "\n")
    csv_df = pd.read_csv(c, header=None, names=col_names)
    # aggregate by hour of day
    csv_df.sort_values(by='hour')
    
    LR2score = []
    LR2fval = np.array([0,0,0,0])
    LR2pval = np.array([0,0,0,0])
    
    LR3score = []
    LR3fval = np.array([0,0,0,0,0,0,0,0])
    LR3pval = np.array([0,0,0,0,0,0,0,0])
    
    LogR2score = []
    LogR2fval = np.array([0,0,0,0])
    LogR2pval = np.array([0,0,0,0])
    
    LogR3score = []
    LogR3fval = np.array([0,0,0,0,0,0,0,0])
    LogR3pval = np.array([0,0,0,0,0,0,0,0])
    
    SVR2score = []
    SVR2fval = np.array([0,0,0,0])
    SVR2pval = np.array([0,0,0,0])
    
    SVR3score = []
    SVR3fval = np.array([0,0,0,0,0,0,0,0])
    SVR3pval = np.array([0,0,0,0,0,0,0,0])
    
    Lasso2score = []
    Lasso2fval = np.array([0,0,0,0])
    Lasso2pval = np.array([0,0,0,0])
    
    Lasso3score = []
    Lasso3fval = np.array([0,0,0,0,0,0,0,0])
    Lasso3pval = np.array([0,0,0,0,0,0,0,0])
    
    for hr in range(0,23):
        #train data setup
        data_prev = csv_df[csv_df['hour'] == hr].copy()
        #features
        data2X_train = data_prev[['n_retweets','sum_followers', 'max_followers', 'hour']].copy()
        data3X_train = data_prev[['n_retweets','sum_followers', 'max_followers', 'hour', 'avg_tweet_per_user', 'n_users', 'n_users_3', 'n_len_100']].copy()
        #targets
        data2Y_train = data_prev[['n_tweets']].copy()
        data3Y_train = data_prev[['n_tweets']].copy()
        
        #test data setup
        data_next = csv_df[csv_df['hour'] == hr+1].copy()
        #features
        data2X_test = data_next[['n_retweets','sum_followers', 'max_followers', 'hour']].copy()
        data3X_test = data_next[['n_retweets','sum_followers', 'max_followers', 'hour', 'avg_tweet_per_user', 'n_users', 'n_users_3', 'n_len_100']].copy()
        #targets
        data2Y_test = data_next[['n_tweets']].copy()
        data3Y_test = data_next[['n_tweets']].copy()
        
        #create models
        # http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html
        LR2model = LR()
        LR3model = LR()
        #http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html
        LogR2model = LogR()
        LogR3model = LogR()
        # http://scikit-learn.org/stable/modules/generated/sklearn.svm.SVR.html
        SVR2model = SVR()
        SVR3model = SVR()
        # http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LassoLars.html
        # Note for whoever writes report: LARS stands for Least Angle Regression
        Lasso2model = lasso()
        Lasso3model = lasso()
        
        #fit models
        LR2model.fit(data2X_train, data2Y_train)
        LR3model.fit(data3X_train, data3Y_train)
        LogR2model.fit(data2X_train, data2Y_train.unstack())
        LogR3model.fit(data3X_train, data3Y_train.unstack())
        SVR2model.fit(data2X_train, data2Y_train.unstack())
        SVR3model.fit(data3X_train, data3Y_train.unstack())
        Lasso2model.fit(data2X_train, data2Y_train)
        Lasso3model.fit(data3X_train, data3Y_train)
        
        #score models using r^2 scoring, f/t value and p value
        # http://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.f_regression.html
        
        #problem 2 Lin Reg
        LR2score.append(LR2model.score(data2X_test, data2Y_test))
        f, p = freg(data2X_test, data2Y_test.unstack())
        LR2fval = LR2fval + f
        LR2pval = LR2pval + p
        
        #problem 3 Lin Reg
        LR3score.append(LR3model.score(data3X_test, data3Y_test))
        f, p = freg(data3X_test, data3Y_test.unstack())
        LR3fval = LR3fval + f
        LR3pval = LR3pval + p
        
        #problem 2 Log Reg
        LogR2score.append(LogR2model.score(data2X_test, data2Y_test.unstack()))
        f, p = freg(data2X_test, data2Y_test.unstack())
        LogR2fval = LogR2fval + f
        LogR2pval = LogR2pval + p
        
        #problem 3 Log Reg
        LogR3score.append(LogR3model.score(data3X_test, data3Y_test.unstack()))
        f, p = freg(data3X_test, data3Y_test.unstack())
        LogR3fval = LogR3fval + f
        LogR3pval = LogR3pval + p
        
        #problem 2 SVR 
        SVR2score.append(SVR2model.score(data2X_test, data2Y_test.unstack()))
        f, p = freg(data2X_test, data2Y_test.unstack())
        SVR2fval = SVR2fval + f
        SVR2pval = SVR2pval + p
        
        #problem 3 SVR 
        SVR3score.append(SVR3model.score(data3X_test, data3Y_test.unstack()))
        f, p = freg(data3X_test, data3Y_test.unstack())
        SVR3fval = SVR3fval + f
        SVR3pval = SVR3pval + p
        
        #problem 2 lARS
        Lasso2score.append(Lasso2model.score(data2X_test, data2Y_test))
        f, p = freg(data2X_test, data2Y_test.unstack())
        Lasso2fval = Lasso2fval + f
        Lasso2pval = Lasso2pval + p
        
        #problem 3 lARS
        Lasso3score.append(Lasso3model.score(data3X_test, data3Y_test))
        f, p = freg(data3X_test, data3Y_test.unstack())
        Lasso3fval = Lasso3fval + f
        Lasso3pval = Lasso3pval + p
    
    # print results
    print("\tLinear Regression Mean Scores:")
    print("\t-Problem 2-")
    print("\t\tR^2 score: %5f" % np.mean(LR2score))
    print("\t\t" + "-"*30)
    for num, f in enumerate(LR2fval):
        print("\t\tT-test feature %d: %9f" % (num+1, f/len(LR2score)))
    print("\t\t" + "-"*30)
    for num, p in enumerate(LR2pval):
        print("\t\tP-value feature %d: %9f" % (num+1, p/len(LR2score)))
    print("\t-Problem 3-")
    print("\t\tR^2 score: %5f" % np.mean(LR3score))
    print("\t\t" + "-"*30)
    for num, f in enumerate(LR3fval):
        print("\t\tT-test feature %d: %9f" % (num+1, f/len(LR3score)))
    print("\t\t" + "-"*30)
    for num, p in enumerate(LR3pval):
        print("\t\tP-value feature %d: %9f" % (num+1, p/len(LR3score)))
    print('\n')
    
    print("\tLogistic Regression Mean Scores:")
    print("\t-Problem 2-")
    print("\t\tR^2 score: %5f" % np.mean(LogR2score))
    print("\t\t" + "-"*30)
    for num, f in enumerate(LogR2fval):
        print("\t\tT-test feature %d: %9f" % (num+1, f/len(LogR2score)))
    print("\t\t" + "-"*30)
    for num, p in enumerate(LogR2pval):
        print("\t\tP-value feature %d: %9f" % (num+1, p/len(LogR2score)))
    print("\t-Problem 3-")
    print("\t\tR^2 score: %5f" % np.mean(LogR3score))
    print("\t\t" + "-"*30)
    for num, f in enumerate(LogR3fval):
        print("\t\tT-test feature %d: %9f" % (num+1, f/len(LogR3score)))
    print("\t\t" + "-"*30)
    for num, p in enumerate(LogR3pval):
        print("\t\tP-value feature %d: %9f" % (num+1, p/len(LogR3score)))
    print('\n')
    
    print("\tSupport Vector Regression Mean Scores:")
    print("\t-Problem 2-")
    print("\t\tR^2 score: %5f" % np.mean(SVR2score))
    print("\t\t" + "-"*30)
    for num, f in enumerate(SVR2fval):
        print("\t\tT-test feature %d: %9f" % (num+1, f/len(SVR2score)))
    print("\t\t" + "-"*30)
    for num, p in enumerate(SVR2pval):
        print("\t\tP-value feature %d: %9f" % (num+1, p/len(SVR2score)))
    print("\t-Problem 3-")
    print("\t\tR^2 score: %5f" % np.mean(SVR3score))
    print("\t\t" + "-"*30)
    for num, f in enumerate(SVR3fval):
        print("\t\tT-test feature %d: %9f" % (num+1, f/len(SVR3score)))
    print("\t\t" + "-"*30)
    for num, p in enumerate(SVR3pval):
        print("\t\tP-value feature %d: %9f" % (num+1, p/len(SVR3score)))
    print('\n')
    
    print("\tLARS Lasso Mean Scores:")
    print("\t-Problem 2-")
    print("\t\tR^2 score: %5f" % np.mean(Lasso2score))
    print("\t\t" + "-"*30)
    for num, f in enumerate(Lasso2fval):
        print("\t\tT-test feature %d: %9f" % (num+1, f/len(Lasso2score)))
    print("\t\t" + "-"*30)
    for num, p in enumerate(Lasso2pval):
        print("\t\tP-value feature %d: %9f" % (num+1, p/len(Lasso2score)))
    print("\t-Problem 3-")
    print("\t\tR^2 score: %5f" % np.mean(Lasso3score))
    print("\t\t" + "-"*30)
    for num, f in enumerate(Lasso3fval):
        print("\t\tT-test feature %d: %9f" % (num+1, f/len(Lasso3score)))
    print("\t\t" + "-"*30)
    for num, p in enumerate(Lasso3pval):
        print("\t\tP-value feature %d: %9f" % (num+1, p/len(Lasso3score)))
    print('\n')
    
    break #remove to run all for all files
    

In [ ]:
len(LR2score)

In [ ]:

a = (np.array([0,0,0,0,0]))
a = a + (np.array([1,2,3,4,5]))
a = a + np.array([1,2,3,4,5])
a = a/2
a